<a href="https://colab.research.google.com/github/szh141/Examples/blob/main/digital_screeni/machine_learning/60_RF_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/bnsreenu/python_for_microscopists/blob/master/060-RandomForest1.py

input data from kaggle
https://www.kaggle.com/code/prashant111/random-forest-classifier-tutorial/input

In [4]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
df = pd.read_csv('/content/drive/MyDrive/Colab images/car_evaluation.csv')

In [5]:
df.rename(columns={'vhigh':'price','vhigh.1':'maitenance_cost','2':'door','2.1':'person','small':'size','unacc':'accept'},inplace=True)

df

,price,maitenance_cost,door,person,size,low,accept
0,vhigh,vhigh,2,2,small,med,unacc
1,vhigh,vhigh,2,2,small,high,unacc
2,vhigh,vhigh,2,2,med,low,unacc
3,vhigh,vhigh,2,2,med,med,unacc
4,vhigh,vhigh,2,2,med,high,unacc
...,...,...,...,...,...,...,...
1722,low,low,5more,more,med,med,good
1723,low,low,5more,more,med,high,vgood
1724,low,low,5more,more,big,low,unacc
1725,low,low,5more,more,big,med,good


df['accept'].value_counts(sort =1)
w/o inplace=True, df['unacc'].value_counts(sort=1)

In [6]:
#Count productivity values to see the split between Good and Bad
sizes = df['accept'].value_counts(sort = 1)
print(sizes)

unacc    1209
acc       384
good       69
vgood      65
Name: accept, dtype: int64


In [7]:
#STEP 2: DROP IRRELEVANT DATA
#In our example, Images_Analyzed reflects whether it is good analysis or bad
#so should not include it. ALso, User number is just a number and has no inflence
#on the productivity, so we can drop it.

df.drop(['size'], axis=1, inplace=True)
df.drop(['low'], axis=1, inplace=True)
df.drop(['maitenance_cost'], axis=1, inplace=True)

df['price'].value_counts(sort=1)

can we do one hot encoding or pd.getdummies ?

In [8]:
#STEP 3: Handle missing values, if needed
#df = df.dropna()  #Drops all rows with at least one null value.


#STEP 4: Convert non-numeric to numeric, if needed.
#Sometimes we may have non-numeric data, for example batch name, user name, city name, etc.
#e.g. if data is in the form of YES and NO then convert to 1 and 2

df['price'][df.price == 'low'] = 1
df['price'][df.price == 'med'] = 2
df['price'][df.price == 'high'] = 3
df['price'][df.price == 'vhigh'] = 4

print(df['price'].value_counts(sort=1))

3    432
2    432
1    432
4    431
Name: price, dtype: int64


In [9]:
df['accept'][df.accept == 'unacc'] = 0
df['accept'][df.accept == 'acc'] = 1
df['accept'][df.accept == 'good'] = 2
df['accept'][df.accept == 'vgood'] = 3

In [10]:
df['door'][df.door == '5more'] = 5
df['person'][df.person == 'more'] = 5

In [19]:
#STEP 5: PREPARE THE DATA.

#Y is the data with dependent variable, this is the accept column
Y = df['accept'].values
Y=Y.astype('int')
# try w/o Y.astype('int'), Y is an object, ValueError


#X is data with independent variables, everything except Productivity column
# Drop label column from X as you don't want that included as one of the features
X = df.drop(['accept'], axis=1)
print(X.head())

  price door person
0     4    2      2
1     4    2      2
2     4    2      2
3     4    2      2
4     4    2      2


In [15]:
#STEP 6: SPLIT THE DATA into TRAIN AND TEST data.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=20)
#random_state can be any integer and it is used as a seed to randomly split dataset.
#By doing this we work with same test dataset evry time, if this is important.
#random_state=None splits dataset randomly every time

#print(X_train)

In [16]:
#STEP 7: Defining the model and training.

# Import the model we are using

# Import the model we are using
#RandomForestRegressor is for regression type of problems.
#For classification we use RandomForestClassifier.
#Both yield similar results except for regressor the result is float
#and for classifier it is an integer.
#Let us use classifier since this is a classification problem

from sklearn.ensemble import RandomForestClassifier
#from sklearn.ensemble import RandomForestRegressor

# Instantiate model with 10 decision trees
model = RandomForestClassifier(n_estimators = 10, random_state = 30)
# Train the model on training data
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=10, random_state=30)

In [21]:
#STEP 8: TESTING THE MODEL BY PREDICTING ON TEST DATA
#AND CALCULATE THE ACCURACY SCORE

prediction_test = model.predict(X_test)
#print(y_test, prediction_test)

from sklearn import metrics
#Print the prediction accuracy
print ("Accuracy = ", metrics.accuracy_score(y_test, prediction_test))
#Test accuracy for various test sizes and see how it gets better with more training data

Accuracy =  0.6859623733719248


In [22]:
#One amazing feature of Random forest is that it provides us info on feature importances
# Get numerical feature importances
#importances = list(model.feature_importances_)

#Let us print them into a nice format.

feature_list = list(X.columns)
feature_imp = pd.Series(model.feature_importances_,index=feature_list).sort_values(ascending=False)
print(feature_imp)

person    0.666889
price     0.223366
door      0.109745
dtype: float64
